In [6]:
import pandas as pd
import glob
import tqdm
import os
from utils.file_utils import open_json

ONLY_ERRORS = False
ONLY_DDG = False
MAX_PROTEIN_SIZE = 600
REMOVE_ERRORS = False
SUBMISSION_DF = False

In [7]:
df = pd.read_csv(
    "./data/main_dataset_creation/outputs/merged/dataset_only_infos.csv")
if SUBMISSION_DF:
    test_df = pd.read_csv("./data/processed_test.csv")
    test_df.head(2)

In [8]:
demask_errors = open_json("errors.json")
if demask_errors != []:
    uniprot_with_errors = list(demask_errors.keys())
else:
    uniprot_with_errors = []
fasta_dir = "./data/main_dataset_creation/DeMaSk_outputs/fasta"
homologs_dir = "./data/main_dataset_creation/DeMaSk_outputs/homologs"
predictions_dir = "./data/main_dataset_creation/DeMaSk_outputs/predictions"
print(uniprot_with_errors)
if REMOVE_ERRORS:
    print(len(glob.glob(f"{fasta_dir}/*.fa")))
    print(len(glob.glob(f"{homologs_dir}/*.a2m")))
    print(len(glob.glob(f"{predictions_dir}/*.txt")))
    for uniprot in uniprot_with_errors:
        for path in glob.glob(f"{fasta_dir}/{uniprot}*"):
            os.remove(path)
        for path in glob.glob(f"{homologs_dir}/{uniprot}*"):
            os.remove(path)
        for path in glob.glob(f"{predictions_dir}/{uniprot}*"):
            os.remove(path)
    print(len(glob.glob(f"{fasta_dir}/*.fa")))
    print(len(glob.glob(f"{homologs_dir}/*.a2m")))
    print(len(glob.glob(f"{predictions_dir}/*.txt")))


[]


In [9]:
unique_df = df[["uniprot", "length", "sequence", "wild_aa", "mutation_position", "mutated_aa", "ddG"]]
print(len(unique_df))
if ONLY_DDG:
    unique_df = unique_df[~(unique_df.ddG.isna())]
unique_df = unique_df[unique_df.length.le(MAX_PROTEIN_SIZE)]
unique_df.drop_duplicates(inplace=True)
print(len(unique_df))

if SUBMISSION_DF:
    unique_test_df = test_df[["uniprot", "sequence",
                                "wild_aa", "mutation_position", "mutated_aa"]]
    print(len(unique_test_df))
    unique_test_df = unique_test_df[~(unique_test_df.mutated_aa.eq('-'))]
    unique_test_df.drop_duplicates(inplace=True)

if ONLY_ERRORS:
    unique_df = unique_df[unique_df.uniprot.isin(uniprot_with_errors)]
    if SUBMISSION_DF:
        unique_test_df = unique_test_df[unique_test_df.uniprot.isin(
            uniprot_with_errors)]

    

print(len(unique_df))
if SUBMISSION_DF:
    print(len(unique_test_df))



def fasta_format_wild(row, files_written):
    sequence = row["sequence"]
    name = row['uniprot']
    fasta_file = f"> {name}\n"+sequence
    with open(f"{fasta_dir}/{name}.fa", "w+") as fp:
        fp.write(fasta_file)
    files_written.append(name)
    return row


def fasta_format_mutations(row, files_written):
    sequence = row["sequence"]
    pos = int(row["mutation_position"])
    sequence = sequence[:pos] + row["mutated_aa"] + sequence[pos+1:]
    name = row['uniprot'] + \
        f"_{row['wild_aa']}{int(row['mutation_position'])}{row['mutated_aa']}"
    fasta_file = f"> {name}\n"+sequence
    with open(f"{fasta_dir}/{name}.fa", "w+") as fp:
        fp.write(fasta_file)
    files_written.append(name)
    return row


files_written = []
unique_df.apply(lambda row: fasta_format_wild(row, files_written), axis=1)
unique_df.apply(lambda row: fasta_format_mutations(row, files_written), axis=1)

if SUBMISSION_DF:
    unique_test_df.apply(lambda row: fasta_format_wild(
        row, files_written), axis=1)
    unique_test_df.apply(lambda row: fasta_format_mutations(
        row, files_written), axis=1)

fasta_files = glob.glob(fasta_dir+"/*.fa")
print(f"there are now {len(fasta_files)} fasta files")

11113
9858
9858
there are now 8605 fasta files


In [6]:
fasta_path = "/home/tom/Documents/Kaggle/novozymes-prediction/data/main_dataset/DeMaSk_scores/fasta/E0WVU4.fa"
uniprot, _ = os.path.splitext(fasta_path)
homologs_path = f"homologs_dir/{uniprot}.a2m"
cmd = f"python3 -m demask.homologs -s {fasta_path} -o {homologs_path}"
print(cmd)

python3 -m demask.homologs -s /home/tom/Documents/Kaggle/novozymes-prediction/data/main_dataset/DeMaSk_scores/fasta/E0WVU4.fa -o ./data/main_dataset/DeMaSk_scores/homologs//home/tom/Documents/Kaggle/novozymes-prediction/data/main_dataset/DeMaSk_scores/fasta/E0WVU4.a2m


In [7]:
# RUN the following VIA THE SCRIPT on a BIG machine

# # create homologs files
# for fasta_path in tqdm.tqdm(fasta_files):
#     uniprot,_ = os.path.splitext(fasta_path) 
#     homologs_path = f"{homologs_dir}/{uniprot}.a2m"
#     cmd = f"python3 -m demask.homologs -s {fasta_path} -o {homologs_path}"
#     os.system(cmd)


In [1]:
import pandas as pd
from utils.features_biopython import add_demask_predictions_by_uniprot

df = pd.read_csv("./data/main_dataset.csv")
df = df[["uniprot", "wild_aa", "mutated_aa", "mutation_position"]]
subdf = df.iloc[:10, :]
uniprot_id = "P06654"

subdf = add_demask_predictions_by_uniprot(subdf, uniprot_id)
subdf


<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1
<class 'pandas.core.frame.DataFrame'> 1


,uniprot,wild_aa,mutated_aa,mutation_position,demask_score,demask_entropy,demask_log2f_var,demask_matrix
0,P06654,M,A,0,-0.3793,0.0015,-17.8728,-0.2641
1,P06654,M,D,0,-0.4907,0.0015,-17.8728,-0.4100
2,P06654,M,E,0,-0.4809,0.0015,-17.8728,-0.3972
3,P06654,M,F,0,-0.3843,0.0015,-17.8728,-0.2707
4,P06654,M,G,0,-0.4602,0.0015,-17.8728,-0.3700
5,P06654,M,H,0,-0.4376,0.0015,-17.8728,-0.3404
6,P06654,M,I,0,-0.2966,0.0015,-17.8728,-0.1558
7,P06654,M,K,0,-0.4418,0.0015,-17.8728,-0.3460
8,P06654,M,L,0,-0.2636,0.0015,-17.8728,-0.1126
9,P06654,T,A,227,-0.1479,1.6708,-18.0952,-0.1608
